In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import os

# URL of the webpage containing the PDF links
url = 'https://www.icar-crida.res.in/publications_annualreports.html'
filenames=[]

def download(pdf_links, download_dir):
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    try:
        for pdf_link in pdf_links:
            # Construct the absolute URL if the link is relative
            if not pdf_link.startswith('http'):
                pdf_link = "https://www.icar-crida.res.in" + pdf_link[1:]
            # Send a GET request to download the PDF file
            pdf_response = requests.get(pdf_link)
            pdf_response.raise_for_status()  # Raise an exception for HTTP errors (e.g., 404)
            # Save the PDF file with a filename based on its URL
            filename = pdf_link.split('/')[-1]
            file_path = os.path.join(download_dir, filename)
            with open(file_path, 'wb') as f:
                f.write(pdf_response.content)
            print(f"Downloaded {filename}")
            filenames.append(filename)
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {pdf_link}: {e}")

def links():
    try:
        # Send a GET request to the webpage
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors (e.g., 404)

        # Parse the HTML content of the webpage
        soup = BeautifulSoup(response.text, 'html.parser')

        tags = soup.find_all('div', class_= 'filter-CRIDA')
        pdf_links=[]
        for tag in tags:
          a = tag.find('a', href=re.compile(r'\.pdf$'))
          pdf_links.append(a['href'])

        print(len(pdf_links))

        # Loop through each filtered PDF link and download the PDF file
        download(pdf_links, "C:\\Users\\DELL\\OneDrive\\Desktop\\LLM2\\data")

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
    return pdf_links

pdf_links = links()